In [6]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [9]:
import hopsworks


os.environ["HOPSWORKS_API_KEY"] = "qf4YnRgKPjj8IeMQ.R4Xeeq4Ge0CVPmW1qLCSOBJLCyqJ9XQoEg3jFnn9WmfPXqO6p5tWvHIGSh1DKnr8"
os.environ["HOPSWORKS_PROJECT_NAME"] = "new_york_taxi"

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

print("API Key:", api_key)
print("Project Name:", project_name)

# Initialize connection to Hopsworks  
project = hopsworks.login(api_key_value=api_key, project=project_name)
print(f"Successfully connected to Hopsworks project: {project_name}")

API Key: qf4YnRgKPjj8IeMQ.R4Xeeq4Ge0CVPmW1qLCSOBJLCyqJ9XQoEg3jFnn9WmfPXqO6p5tWvHIGSh1DKnr8
Project Name: new_york_taxi
2025-03-03 09:31:35,885 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 09:31:35,889 INFO: Initializing external client
2025-03-03 09:31:35,889 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 09:31:36,646 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Successfully connected to Hopsworks project: new_york_taxi


In [10]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-03 09:31:37,835 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 09:31:37,840 INFO: Initializing external client
2025-03-03 09:31:37,841 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 09:31:38,639 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Fetching data from 2025-02-02 14:31:37.833131+00:00 to 2025-03-03 13:31:37.833131+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.12s) 


In [11]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 09:31:54,021 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 09:31:54,025 INFO: Initializing external client
2025-03-03 09:31:54,026 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 09:31:56,034 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [12]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [13]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,255,2.0,2025-03-03 15:00:00+00:00
1,218,1.0,2025-03-03 15:00:00+00:00
2,237,276.0,2025-03-03 15:00:00+00:00
3,57,0.0,2025-03-03 15:00:00+00:00
4,56,0.0,2025-03-03 15:00:00+00:00
...,...,...,...
246,187,0.0,2025-03-03 15:00:00+00:00
247,71,0.0,2025-03-03 15:00:00+00:00
248,8,0.0,2025-03-03 15:00:00+00:00
249,168,0.0,2025-03-03 15:00:00+00:00


In [14]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-03 09:31:59,432 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 09:31:59,436 INFO: Initializing external client
2025-03-03 09:31:59,436 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 09:32:00,447 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677


In [15]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214677/fs/1202308/fg/1401852


Uploading Dataframe: 100.00% |██████████| Rows 251/251 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214677/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)